### Team Members:

1- Abanoub Samir Girgis | ID: 20190001

2- Aram Gamal Youssef     | ID: 20190083

In [1]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from tqdm import tqdm
from scipy.spatial import distance
from numpy.linalg import norm

2022-11-29 09:28:56.746533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 09:28:58.120609: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64::/home/abanoub/anaconda3/lib/:/home/abanoub/anaconda3/lib/
2022-11-29 09:28:58.120738: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/li

In [2]:
doc1 = pd.read_excel('pca.xlsx')
doc2 = pd.read_excel('dimensionality_redution.xlsx')
doc3 = pd.read_excel('food.xlsx')
print(len(doc1))
print(len(doc2))
print(len(doc3))

5
7
4


In [3]:
def text_preprocessing(
        text: list,
        punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
        stop_words=['and', 'a', 'is', 'the', 'in','andor', 'be','he','their','usually','within','useds','unique','used','will', 'an','was', 'but', 'this', 'were', 'with', 'of', 'also','on', '.', 'for', 'any', 'its', 'and', 'are', 'from', 'both', 'as','or','into','some','that','so','to','can','such','other','has','per','while','by','most','where','than','use','two']
) -> list:
    """
    A method to preproces text
    """
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x != '']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text



In [4]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc1 = []
all_text_doc1 = []
for text in doc1['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc1 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc1.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc1.append([word] + [text[(i - w - 1)]])
                
word_lists_doc1

['principal', 'component', 'analysis', 'pca', 'popular', 'technique', 'analyzing', 'large', 'datasets', 'containing', 'high', 'number', 'dimensionsfeatures', 'observation']
['increasing', 'interpretability', 'data', 'preserving', 'maximum', 'amount', 'information', 'enabling', 'visualization', 'multidimensional', 'data', 'formally']
['pca', 'statistical', 'technique', 'reducing', 'dimensionality', 'dataset', 'accomplished', 'linearly', 'transforming', 'data', 'new', 'coordinate', 'system', 'variation', 'data', 'described', 'fewer', 'dimensions', 'initial', 'data']
['many', 'studies', 'first', 'principal', 'components', 'order', 'plot', 'data', 'dimensions', 'visually', 'identify', 'clusters', 'closely', 'related', 'data', 'points']
['principal', 'component', 'analysis', 'applications', 'many', 'fields', 'population', 'genetics', 'microbiome', 'studies', 'atmospheric', 'science']


[['principal', 'component'],
 ['principal', 'analysis'],
 ['component', 'analysis'],
 ['component', 'principal'],
 ['component', 'pca'],
 ['analysis', 'pca'],
 ['analysis', 'component'],
 ['analysis', 'popular'],
 ['analysis', 'principal'],
 ['pca', 'popular'],
 ['pca', 'analysis'],
 ['pca', 'technique'],
 ['pca', 'component'],
 ['popular', 'technique'],
 ['popular', 'pca'],
 ['popular', 'analyzing'],
 ['popular', 'analysis'],
 ['technique', 'analyzing'],
 ['technique', 'popular'],
 ['technique', 'large'],
 ['technique', 'pca'],
 ['analyzing', 'large'],
 ['analyzing', 'technique'],
 ['analyzing', 'datasets'],
 ['analyzing', 'popular'],
 ['large', 'datasets'],
 ['large', 'analyzing'],
 ['large', 'containing'],
 ['large', 'technique'],
 ['datasets', 'containing'],
 ['datasets', 'large'],
 ['datasets', 'high'],
 ['datasets', 'analyzing'],
 ['containing', 'high'],
 ['containing', 'datasets'],
 ['containing', 'number'],
 ['containing', 'large'],
 ['high', 'number'],
 ['high', 'containing'],

In [5]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc2 = []
all_text_doc2 = []
for text in doc2['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc2 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc2.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc2.append([word] + [text[(i - w - 1)]])
                
word_lists_doc2

['dimensionality', 'reduction', 'dimension', 'reduction', 'transformation', 'data', 'highdimensional', 'space', 'lowdimensional', 'space', 'lowdimensional', 'representation', 'retains', 'meaningful', 'properties', 'original', 'data']
['ideally', 'close', 'intrinsic', 'dimension', 'working', 'highdimensional', 'spaces', 'undesirable', 'many', 'reasons', 'raw', 'data', 'often', 'sparse', 'consequence', 'curse', 'dimensionality']
['analyzing', 'data', 'computationally', 'intractable', 'hard', 'control', 'deal']
['dimensionality', 'reduction', 'common', 'fields', 'deal', 'large', 'numbers', 'observations', 'large', 'numbers', 'variables']
['signal', 'processing', 'speech', 'recognition', 'neuroinformatics', 'bioinformatics', 'methods', 'commonly', 'divided', 'linear', 'nonlinear', 'approaches']
['approaches', 'divided', 'feature', 'selection', 'feature', 'extraction']
['dimensionality', 'reduction', 'noise', 'reduction', 'data', 'visualization', 'cluster', 'analysis', 'intermediate', 'step

[['dimensionality', 'reduction'],
 ['dimensionality', 'dimension'],
 ['reduction', 'dimension'],
 ['reduction', 'dimensionality'],
 ['reduction', 'reduction'],
 ['dimension', 'reduction'],
 ['dimension', 'reduction'],
 ['dimension', 'transformation'],
 ['dimension', 'dimensionality'],
 ['reduction', 'transformation'],
 ['reduction', 'dimension'],
 ['reduction', 'data'],
 ['reduction', 'reduction'],
 ['transformation', 'data'],
 ['transformation', 'reduction'],
 ['transformation', 'highdimensional'],
 ['transformation', 'dimension'],
 ['data', 'highdimensional'],
 ['data', 'transformation'],
 ['data', 'space'],
 ['data', 'reduction'],
 ['highdimensional', 'space'],
 ['highdimensional', 'data'],
 ['highdimensional', 'lowdimensional'],
 ['highdimensional', 'transformation'],
 ['space', 'lowdimensional'],
 ['space', 'highdimensional'],
 ['space', 'space'],
 ['space', 'data'],
 ['lowdimensional', 'space'],
 ['lowdimensional', 'space'],
 ['lowdimensional', 'lowdimensional'],
 ['lowdimensiona

In [6]:
# Defining the window for context
window = 2

# Creating a placeholder for the scanning of the word list
word_lists_doc3 = []
all_text_doc3 = []
for text in doc3['text']:

    # Cleaning the text
    text = text_preprocessing(text)
    print (text)
    
    # Appending to the all text list
    all_text_doc3 += text

    # Creating a context dictionary
    for i, word in enumerate(text):
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(text):
                word_lists_doc3.append([word] + [text[(i + 1 + w)]])
            # Getting the context that is behind by *window* words
            if i - w - 1 >= 0:
                word_lists_doc3.append([word] + [text[(i - w - 1)]])
                
word_lists_doc3

['food', 'substance', 'consumed', 'organism', 'nutritional', 'support']
['food', 'plant', 'animal', 'fungal', 'origin', 'contains', 'essential', 'nutrients', 'carbohydrates', 'fats', 'proteins', 'vitamins', 'minerals']
['substance', 'ingested', 'organism', 'assimilated', 'organisms', 'cells', 'provide', 'energy', 'maintain', 'life', 'stimulate', 'growth']
['different', 'species', 'animals', 'have', 'different', 'feeding', 'behaviours', 'satisfy', 'needs', 'metabolisms', 'often', 'evolved', 'fill', 'specific', 'ecological', 'niche', 'specific', 'geographical', 'contexts']


[['food', 'substance'],
 ['food', 'consumed'],
 ['substance', 'consumed'],
 ['substance', 'food'],
 ['substance', 'organism'],
 ['consumed', 'organism'],
 ['consumed', 'substance'],
 ['consumed', 'nutritional'],
 ['consumed', 'food'],
 ['organism', 'nutritional'],
 ['organism', 'consumed'],
 ['organism', 'support'],
 ['organism', 'substance'],
 ['nutritional', 'support'],
 ['nutritional', 'organism'],
 ['nutritional', 'consumed'],
 ['support', 'nutritional'],
 ['support', 'organism'],
 ['food', 'plant'],
 ['food', 'animal'],
 ['plant', 'animal'],
 ['plant', 'food'],
 ['plant', 'fungal'],
 ['animal', 'fungal'],
 ['animal', 'plant'],
 ['animal', 'origin'],
 ['animal', 'food'],
 ['fungal', 'origin'],
 ['fungal', 'animal'],
 ['fungal', 'contains'],
 ['fungal', 'plant'],
 ['origin', 'contains'],
 ['origin', 'fungal'],
 ['origin', 'essential'],
 ['origin', 'animal'],
 ['contains', 'essential'],
 ['contains', 'origin'],
 ['contains', 'nutrients'],
 ['contains', 'fungal'],
 ['essential', 'nutr

In [7]:
def create_unique_word_dict(text: list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict


In [8]:
unique_word_dict1 = create_unique_word_dict(all_text_doc1)
unique_word_dict2 = create_unique_word_dict(all_text_doc2)
unique_word_dict3 = create_unique_word_dict(all_text_doc3)

# Defining the number of features (unique words)
n_words_doc1 = len(unique_word_dict1)
n_words_doc2 = len(unique_word_dict2)
n_words_doc3 = len(unique_word_dict3)

print(n_words_doc1)
print(n_words_doc2)
print(n_words_doc3)


59
61
45


In [9]:
unique_word_dict1

{'accomplished': 0,
 'amount': 1,
 'analysis': 2,
 'analyzing': 3,
 'applications': 4,
 'atmospheric': 5,
 'closely': 6,
 'clusters': 7,
 'component': 8,
 'components': 9,
 'containing': 10,
 'coordinate': 11,
 'data': 12,
 'dataset': 13,
 'datasets': 14,
 'described': 15,
 'dimensionality': 16,
 'dimensions': 17,
 'dimensionsfeatures': 18,
 'enabling': 19,
 'fewer': 20,
 'fields': 21,
 'first': 22,
 'formally': 23,
 'genetics': 24,
 'high': 25,
 'identify': 26,
 'increasing': 27,
 'information': 28,
 'initial': 29,
 'interpretability': 30,
 'large': 31,
 'linearly': 32,
 'many': 33,
 'maximum': 34,
 'microbiome': 35,
 'multidimensional': 36,
 'new': 37,
 'number': 38,
 'observation': 39,
 'order': 40,
 'pca': 41,
 'plot': 42,
 'points': 43,
 'popular': 44,
 'population': 45,
 'preserving': 46,
 'principal': 47,
 'reducing': 48,
 'related': 49,
 'science': 50,
 'statistical': 51,
 'studies': 52,
 'system': 53,
 'technique': 54,
 'transforming': 55,
 'variation': 56,
 'visualization': 5

In [10]:
unique_word_dict2

{'analyses': 0,
 'analysis': 1,
 'analyzing': 2,
 'approaches': 3,
 'bioinformatics': 4,
 'close': 5,
 'cluster': 6,
 'common': 7,
 'commonly': 8,
 'computationally': 9,
 'consequence': 10,
 'control': 11,
 'curse': 12,
 'data': 13,
 'deal': 14,
 'dimension': 15,
 'dimensionality': 16,
 'divided': 17,
 'extraction': 18,
 'facilitate': 19,
 'feature': 20,
 'fields': 21,
 'hard': 22,
 'highdimensional': 23,
 'ideally': 24,
 'intermediate': 25,
 'intractable': 26,
 'intrinsic': 27,
 'large': 28,
 'linear': 29,
 'lowdimensional': 30,
 'many': 31,
 'meaningful': 32,
 'methods': 33,
 'neuroinformatics': 34,
 'noise': 35,
 'nonlinear': 36,
 'numbers': 37,
 'observations': 38,
 'often': 39,
 'original': 40,
 'processing': 41,
 'properties': 42,
 'raw': 43,
 'reasons': 44,
 'recognition': 45,
 'reduction': 46,
 'representation': 47,
 'retains': 48,
 'selection': 49,
 'signal': 50,
 'space': 51,
 'spaces': 52,
 'sparse': 53,
 'speech': 54,
 'step': 55,
 'transformation': 56,
 'undesirable': 57,


In [11]:
unique_word_dict3

{'animal': 0,
 'animals': 1,
 'assimilated': 2,
 'behaviours': 3,
 'carbohydrates': 4,
 'cells': 5,
 'consumed': 6,
 'contains': 7,
 'contexts': 8,
 'different': 9,
 'ecological': 10,
 'energy': 11,
 'essential': 12,
 'evolved': 13,
 'fats': 14,
 'feeding': 15,
 'fill': 16,
 'food': 17,
 'fungal': 18,
 'geographical': 19,
 'growth': 20,
 'have': 21,
 'ingested': 22,
 'life': 23,
 'maintain': 24,
 'metabolisms': 25,
 'minerals': 26,
 'needs': 27,
 'niche': 28,
 'nutrients': 29,
 'nutritional': 30,
 'often': 31,
 'organism': 32,
 'organisms': 33,
 'origin': 34,
 'plant': 35,
 'proteins': 36,
 'provide': 37,
 'satisfy': 38,
 'species': 39,
 'specific': 40,
 'stimulate': 41,
 'substance': 42,
 'support': 43,
 'vitamins': 44}

In [12]:
all_text=all_text_doc1+all_text_doc2+all_text_doc3

unique_word_dict = create_unique_word_dict(all_text)
# Defining the number of features (unique words)
n_words = len(unique_word_dict)
print(n_words)
unique_word_dict

156


{'accomplished': 0,
 'amount': 1,
 'analyses': 2,
 'analysis': 3,
 'analyzing': 4,
 'animal': 5,
 'animals': 6,
 'applications': 7,
 'approaches': 8,
 'assimilated': 9,
 'atmospheric': 10,
 'behaviours': 11,
 'bioinformatics': 12,
 'carbohydrates': 13,
 'cells': 14,
 'close': 15,
 'closely': 16,
 'cluster': 17,
 'clusters': 18,
 'common': 19,
 'commonly': 20,
 'component': 21,
 'components': 22,
 'computationally': 23,
 'consequence': 24,
 'consumed': 25,
 'containing': 26,
 'contains': 27,
 'contexts': 28,
 'control': 29,
 'coordinate': 30,
 'curse': 31,
 'data': 32,
 'dataset': 33,
 'datasets': 34,
 'deal': 35,
 'described': 36,
 'different': 37,
 'dimension': 38,
 'dimensionality': 39,
 'dimensions': 40,
 'dimensionsfeatures': 41,
 'divided': 42,
 'ecological': 43,
 'enabling': 44,
 'energy': 45,
 'essential': 46,
 'evolved': 47,
 'extraction': 48,
 'facilitate': 49,
 'fats': 50,
 'feature': 51,
 'feeding': 52,
 'fewer': 53,
 'fields': 54,
 'fill': 55,
 'first': 56,
 'food': 57,
 'f

In [13]:
word_lists=word_lists_doc1+word_lists_doc2+word_lists_doc3
word_lists

[['principal', 'component'],
 ['principal', 'analysis'],
 ['component', 'analysis'],
 ['component', 'principal'],
 ['component', 'pca'],
 ['analysis', 'pca'],
 ['analysis', 'component'],
 ['analysis', 'popular'],
 ['analysis', 'principal'],
 ['pca', 'popular'],
 ['pca', 'analysis'],
 ['pca', 'technique'],
 ['pca', 'component'],
 ['popular', 'technique'],
 ['popular', 'pca'],
 ['popular', 'analyzing'],
 ['popular', 'analysis'],
 ['technique', 'analyzing'],
 ['technique', 'popular'],
 ['technique', 'large'],
 ['technique', 'pca'],
 ['analyzing', 'large'],
 ['analyzing', 'technique'],
 ['analyzing', 'datasets'],
 ['analyzing', 'popular'],
 ['large', 'datasets'],
 ['large', 'analyzing'],
 ['large', 'containing'],
 ['large', 'technique'],
 ['datasets', 'containing'],
 ['datasets', 'large'],
 ['datasets', 'high'],
 ['datasets', 'analyzing'],
 ['containing', 'high'],
 ['containing', 'datasets'],
 ['containing', 'number'],
 ['containing', 'large'],
 ['high', 'number'],
 ['high', 'containing'],

In [14]:
# Getting all the unique words
words = list(unique_word_dict.keys())
print(words)

# Creating the X and Y matrices using one hot encoding
print(n_words)
X = []
Y = []
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    print(word_list)
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])
    # print (word_list)
    print(word_list[0], main_word_index)
    print(word_list[1], context_word_index)

    # Creating the placeholders
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

# Converting the matrices into a sparse format because the vast majority of the data are 0s
print(X)
print(Y)


['accomplished', 'amount', 'analyses', 'analysis', 'analyzing', 'animal', 'animals', 'applications', 'approaches', 'assimilated', 'atmospheric', 'behaviours', 'bioinformatics', 'carbohydrates', 'cells', 'close', 'closely', 'cluster', 'clusters', 'common', 'commonly', 'component', 'components', 'computationally', 'consequence', 'consumed', 'containing', 'contains', 'contexts', 'control', 'coordinate', 'curse', 'data', 'dataset', 'datasets', 'deal', 'described', 'different', 'dimension', 'dimensionality', 'dimensions', 'dimensionsfeatures', 'divided', 'ecological', 'enabling', 'energy', 'essential', 'evolved', 'extraction', 'facilitate', 'fats', 'feature', 'feeding', 'fewer', 'fields', 'fill', 'first', 'food', 'formally', 'fungal', 'genetics', 'geographical', 'growth', 'hard', 'have', 'high', 'highdimensional', 'ideally', 'identify', 'increasing', 'information', 'ingested', 'initial', 'intermediate', 'interpretability', 'intractable', 'intrinsic', 'large', 'life', 'linear', 'linearly', '

728it [00:00, 40497.27it/s]

['principal', 'component']
principal 116
component 21
['principal', 'analysis']
principal 116
analysis 3
['component', 'analysis']
component 21
analysis 3
['component', 'principal']
component 21
principal 116
['component', 'pca']
component 21
pca 109
['analysis', 'pca']
analysis 3
pca 109
['analysis', 'component']
analysis 3
component 21
['analysis', 'popular']
analysis 3
popular 113
['analysis', 'principal']
analysis 3
principal 116
['pca', 'popular']
pca 109
popular 113
['pca', 'analysis']
pca 109
analysis 3
['pca', 'technique']
pca 109
technique 146
['pca', 'component']
pca 109
component 21
['popular', 'technique']
popular 113
technique 146
['popular', 'pca']
popular 113
pca 109
['popular', 'analyzing']
popular 113
analyzing 4
['popular', 'analysis']
popular 113
analysis 3
['technique', 'analyzing']
technique 146
analyzing 4
['technique', 'popular']
technique 146
popular 113
['technique', 'large']
technique 146
large 77
['technique', 'pca']
technique 146
pca 109
['analyzing', 'large

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [15]:
XX = tf.convert_to_tensor(X, dtype=tf.float32)
YY = tf.convert_to_tensor(Y, dtype=tf.float32)
print(XX.shape)
print(YY.shape)


(728, 156)
(728, 156)


2022-11-29 09:29:03.797041: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-11-29 09:29:03.797105: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: abanoub-G3-3590
2022-11-29 09:29:03.797116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: abanoub-G3-3590
2022-11-29 09:29:03.797246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-11-29 09:29:03.797292: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-11-29 09:29:03.797314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 470.141.3 does not match DSO version 515.65.1 -- cannot find working devices in this configuration
20

In [16]:
def CreateModel():
    # Defining the size of the embedding
    embed_size = 2
    # Defining the neural network

    # inp = Input(shape=(X.shape[1],))
    inp = Input(shape=XX.shape[1])  # 21
    x = Dense(units=embed_size, activation='linear')(inp)
    # x = Dense(units=21, activation='softmax')(x)
    x = Dense(units=YY.shape[1], activation='softmax')(x)

    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.summary()
    return model


In [18]:
model = CreateModel()
# Optimizing the network weights
model.fit(x=XX,y=YY,batch_size=10,epochs=2000)
# The input layer
weights = model.get_weights()[0]

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 156)]             0         
                                                                 
 dense_2 (Dense)             (None, 2)                 314       
                                                                 
 dense_3 (Dense)             (None, 156)               468       
                                                                 
Total params: 782
Trainable params: 782
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2000
73/73 [==============================] - 0s 1ms/step - loss: 5.0502
Epoch 2/2000
73/73 [==============================] - 0s 1ms/step - loss: 5.0417
Epoch 3/2000
73/73 [==============================] - 0s 1ms/step - loss: 5.0350
Epoch 4/2000
73/73 [==============================] - 0s 1ms/step - loss: 5.0281
E

73/73 [==============================] - 0s 1ms/step - loss: 4.0194
Epoch 92/2000
73/73 [==============================] - 0s 1ms/step - loss: 4.0122
Epoch 93/2000
73/73 [==============================] - 0s 1ms/step - loss: 4.0045
Epoch 94/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9971
Epoch 95/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9897
Epoch 96/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9824
Epoch 97/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9751
Epoch 98/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9681
Epoch 99/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9612
Epoch 100/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9544
Epoch 101/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9477
Epoch 102/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.9414
Epoch 103/2000
73/73 [=====

Epoch 190/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5525
Epoch 191/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5495
Epoch 192/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5466
Epoch 193/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5435
Epoch 194/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5413
Epoch 195/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5382
Epoch 196/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5352
Epoch 197/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5323
Epoch 198/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5291
Epoch 199/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5263
Epoch 200/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5237
Epoch 201/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.5210
Epoc

73/73 [==============================] - 0s 1ms/step - loss: 3.3359
Epoch 289/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3339
Epoch 290/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3329
Epoch 291/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3307
Epoch 292/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3290
Epoch 293/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3278
Epoch 294/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3261
Epoch 295/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3247
Epoch 296/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3227
Epoch 297/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3212
Epoch 298/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3195
Epoch 299/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.3181
Epoch 300/2000
73/7

73/73 [==============================] - 0s 1ms/step - loss: 3.1983
Epoch 387/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1976
Epoch 388/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1963
Epoch 389/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1952
Epoch 390/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1942
Epoch 391/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1925
Epoch 392/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1912
Epoch 393/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1906
Epoch 394/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1891
Epoch 395/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1880
Epoch 396/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1879
Epoch 397/2000
73/73 [==============================] - 0s 1ms/step - loss: 3.1860
Epoch 398/2000
73/7

...

73/73 [==============================] - 0s 1ms/step - loss: 2.7315
Epoch 1558/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7319
Epoch 1559/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7326
Epoch 1560/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7324
Epoch 1561/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7326
Epoch 1562/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7319
Epoch 1563/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7316
Epoch 1564/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7316
Epoch 1565/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7324
Epoch 1566/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7324
Epoch 1567/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7315
Epoch 1568/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7303
Epoch 15

73/73 [==============================] - 0s 1ms/step - loss: 2.7197
Epoch 1655/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7210
Epoch 1656/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7199
Epoch 1657/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7194
Epoch 1658/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7203
Epoch 1659/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7205
Epoch 1660/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7195
Epoch 1661/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7197
Epoch 1662/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7186
Epoch 1663/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7192
Epoch 1664/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7181
Epoch 1665/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7185
Epoch 16

73/73 [==============================] - 0s 1ms/step - loss: 2.7090
Epoch 1752/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7092
Epoch 1753/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7092
Epoch 1754/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7087
Epoch 1755/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7085
Epoch 1756/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7088
Epoch 1757/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7089
Epoch 1758/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7075
Epoch 1759/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7069
Epoch 1760/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7085
Epoch 1761/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7084
Epoch 1762/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.7078
Epoch 17

73/73 [==============================] - 0s 1ms/step - loss: 2.6983
Epoch 1849/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6979
Epoch 1850/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6977
Epoch 1851/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6979
Epoch 1852/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6980
Epoch 1853/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6977
Epoch 1854/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6977
Epoch 1855/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6972
Epoch 1856/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6982
Epoch 1857/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6977
Epoch 1858/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6964
Epoch 1859/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6970
Epoch 18

73/73 [==============================] - 0s 1ms/step - loss: 2.6893
Epoch 1946/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6882
Epoch 1947/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6893
Epoch 1948/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6876
Epoch 1949/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6880
Epoch 1950/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6885
Epoch 1951/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6871
Epoch 1952/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6876
Epoch 1953/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6879
Epoch 1954/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6867
Epoch 1955/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6884
Epoch 1956/2000
73/73 [==============================] - 0s 1ms/step - loss: 2.6879
Epoch 19

In [20]:
print(weights.shape)
print(weights)

(156, 2)
[[ 9.75962830e+00 -4.13038194e-01]
 [-1.28305936e+00 -2.13645101e-01]
 [ 5.17520487e-01 -3.56740499e+00]
 [ 2.97712898e+00 -2.22178411e+00]
 [ 1.35417318e+00 -1.87441707e+00]
 [-3.84677172e+00 -4.24409914e+00]
 [-1.34033842e+01  2.07186127e+01]
 [ 5.51596165e+00 -3.22952771e+00]
 [-1.46290436e+01  4.00604486e+00]
 [-5.87285566e+00 -1.23711834e+01]
 [ 3.61666656e+00 -3.71518016e+00]
 [-5.36000395e+00  1.00526247e+01]
 [-2.79275298e+00  2.54817128e+00]
 [-1.02998953e+01 -2.71150327e+00]
 [-8.63107300e+00 -1.32826862e+01]
 [ 1.50466089e+01  4.14909267e+00]
 [ 8.58348966e-01  4.92146683e+00]
 [ 1.44859374e+00 -9.49319303e-01]
 [ 6.06025755e-01  7.04675674e+00]
 [ 3.11207867e+00 -1.64554071e+00]
 [-5.59975100e+00  2.25641537e+00]
 [ 9.43167114e+00 -4.37645197e+00]
 [ 2.80079079e+00 -9.62854147e-01]
 [ 2.70948559e-01 -1.09312391e+00]
 [ 3.21939677e-01  3.68141234e-01]
 [-3.58706641e+00 -1.14415045e+01]
 [ 1.13094723e+00 -1.00379124e+01]
 [-7.71473885e+00 -4.43269205e+00]
 [-1.770013

In [21]:
# get the weight for each unique word
embedding_dict = {}
for word in words:  # to pick the row of weight of two values for each unique word since weights = 21*2
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
    })

print(embedding_dict)

{'accomplished': array([ 9.759628 , -0.4130382], dtype=float32), 'amount': array([-1.2830594, -0.2136451], dtype=float32), 'analyses': array([ 0.5175205, -3.567405 ], dtype=float32), 'analysis': array([ 2.977129, -2.221784], dtype=float32), 'analyzing': array([ 1.3541732, -1.8744171], dtype=float32), 'animal': array([-3.8467717, -4.244099 ], dtype=float32), 'animals': array([-13.403384,  20.718613], dtype=float32), 'applications': array([ 5.5159616, -3.2295277], dtype=float32), 'approaches': array([-14.629044,   4.006045], dtype=float32), 'assimilated': array([ -5.8728557, -12.371183 ], dtype=float32), 'atmospheric': array([ 3.6166666, -3.7151802], dtype=float32), 'behaviours': array([-5.360004, 10.052625], dtype=float32), 'bioinformatics': array([-2.792753 ,  2.5481713], dtype=float32), 'carbohydrates': array([-10.299895 ,  -2.7115033], dtype=float32), 'cells': array([ -8.631073, -13.282686], dtype=float32), 'close': array([15.046609 ,  4.1490927], dtype=float32), 'closely': array([0.

In [22]:
# get the weight for each unique word
for word in unique_word_dict1: 
    unique_word_dict1[word]=embedding_dict[word]
for word in unique_word_dict2: 
    unique_word_dict2[word]=embedding_dict[word]
for word in unique_word_dict3: 
    unique_word_dict3[word]=embedding_dict[word]

In [23]:
unique_word_dict1

{'accomplished': array([ 9.759628 , -0.4130382], dtype=float32),
 'amount': array([-1.2830594, -0.2136451], dtype=float32),
 'analysis': array([ 2.977129, -2.221784], dtype=float32),
 'analyzing': array([ 1.3541732, -1.8744171], dtype=float32),
 'applications': array([ 5.5159616, -3.2295277], dtype=float32),
 'atmospheric': array([ 3.6166666, -3.7151802], dtype=float32),
 'closely': array([0.85834897, 4.921467  ], dtype=float32),
 'clusters': array([0.60602576, 7.0467567 ], dtype=float32),
 'component': array([ 9.431671, -4.376452], dtype=float32),
 'components': array([ 2.8007908 , -0.96285415], dtype=float32),
 'containing': array([  1.1309472, -10.037912 ], dtype=float32),
 'coordinate': array([4.103363 , 3.7256508], dtype=float32),
 'data': array([0.4674042, 0.5269708], dtype=float32),
 'dataset': array([19.899216, -1.95408 ], dtype=float32),
 'datasets': array([ 1.7816225, -8.2686615], dtype=float32),
 'described': array([2.1316164, 3.6607199], dtype=float32),
 'dimensionality': a

In [24]:
unique_word_dict2

{'analyses': array([ 0.5175205, -3.567405 ], dtype=float32),
 'analysis': array([ 2.977129, -2.221784], dtype=float32),
 'analyzing': array([ 1.3541732, -1.8744171], dtype=float32),
 'approaches': array([-14.629044,   4.006045], dtype=float32),
 'bioinformatics': array([-2.792753 ,  2.5481713], dtype=float32),
 'close': array([15.046609 ,  4.1490927], dtype=float32),
 'cluster': array([ 1.4485937, -0.9493193], dtype=float32),
 'common': array([ 3.1120787, -1.6455407], dtype=float32),
 'commonly': array([-5.599751 ,  2.2564154], dtype=float32),
 'computationally': array([ 0.27094856, -1.0931239 ], dtype=float32),
 'consequence': array([0.32193968, 0.36814123], dtype=float32),
 'control': array([-0.32660654, -2.592645  ], dtype=float32),
 'curse': array([1.4537807 , 0.00414374], dtype=float32),
 'data': array([0.4674042, 0.5269708], dtype=float32),
 'deal': array([ 0.95846784, -2.6847208 ], dtype=float32),
 'dimension': array([11.246129 ,  1.6530478], dtype=float32),
 'dimensionality': a

In [25]:
unique_word_dict3

{'animal': array([-3.8467717, -4.244099 ], dtype=float32),
 'animals': array([-13.403384,  20.718613], dtype=float32),
 'assimilated': array([ -5.8728557, -12.371183 ], dtype=float32),
 'behaviours': array([-5.360004, 10.052625], dtype=float32),
 'carbohydrates': array([-10.299895 ,  -2.7115033], dtype=float32),
 'cells': array([ -8.631073, -13.282686], dtype=float32),
 'consumed': array([ -3.5870664, -11.4415045], dtype=float32),
 'contains': array([-7.714739, -4.432692], dtype=float32),
 'contexts': array([-17.700138,  15.377317], dtype=float32),
 'different': array([-6.659889, 17.127745], dtype=float32),
 'ecological': array([-11.032858,   9.149593], dtype=float32),
 'energy': array([-10.078145, -11.073633], dtype=float32),
 'essential': array([-10.594704 ,  -4.5750732], dtype=float32),
 'evolved': array([-3.2354445,  3.4004436], dtype=float32),
 'fats': array([-7.3385615, -1.8023442], dtype=float32),
 'feeding': array([-7.213373, 12.244213], dtype=float32),
 'fill': array([-4.29132

# Approach 1 
### Calculate the Euclidean distance between each word in the first document and every word in the second. 

In [26]:
from scipy.spatial import distance
min_dis1=[]
distance_similarity1=0
for word_doc1 in unique_word_dict1:
    word1=''
    word2=''
    minimum=100000000.0
    for word_doc2 in unique_word_dict2:
        x=distance.euclidean(unique_word_dict1[word_doc1],unique_word_dict2[word_doc2]) 
        if x < minimum:
            minimum=x
            word1=word_doc1
            word2=word_doc2
    distance_similarity1+=minimum
    min_dis1.append((word1,word2,minimum))
min_dis1

[('accomplished', 'spaces', 2.180652141571045),
 ('amount', 'intractable', 1.470145344734192),
 ('analysis', 'analysis', 0.0),
 ('analyzing', 'analyzing', 0.0),
 ('applications', 'dimensionality', 2.6957974433898926),
 ('atmospheric', 'analysis', 1.624573826789856),
 ('closely', 'recognition', 3.25799822807312),
 ('clusters', 'recognition', 2.8217687606811523),
 ('component', 'dimensionality', 5.239199638366699),
 ('components', 'many', 0.4346756339073181),
 ('containing', 'large', 3.2015469074249268),
 ('coordinate', 'original', 3.3397412300109863),
 ('data', 'data', 0.0),
 ('dataset', 'ideally', 5.802842617034912),
 ('datasets', 'large', 2.7483086585998535),
 ('described', 'sparse', 3.350501298904419),
 ('dimensionality', 'dimensionality', 0.0),
 ('dimensions', 'sparse', 2.0321309566497803),
 ('dimensionsfeatures', 'large', 4.752598762512207),
 ('enabling', 'intractable', 0.8016043305397034),
 ('fewer', 'sparse', 2.49094295501709),
 ('fields', 'fields', 0.0),
 ('first', 'common', 1.7

In [27]:
min_dis2=[]
distance_similarity2=0
for word_doc1 in unique_word_dict1:
    word1=''
    word2=''
    minimum=100000000.0
    for word_doc3 in unique_word_dict3:
        x=distance.euclidean(unique_word_dict1[word_doc1],unique_word_dict3[word_doc3]) 
        if x<minimum:
            minimum=x
            word1=word_doc1
            word2=word_doc2
    distance_similarity2 +=minimum
    min_dis2.append((word1,word2,minimum))
min_dis2

[('accomplished', 'working', 10.615436553955078),
 ('amount', 'working', 1.7477689981460571),
 ('analysis', 'working', 5.179652214050293),
 ('analyzing', 'working', 3.8894870281219482),
 ('applications', 'working', 7.7650556564331055),
 ('atmospheric', 'working', 6.713937282562256),
 ('closely', 'working', 3.426067352294922),
 ('clusters', 'working', 4.080612659454346),
 ('component', 'working', 11.673550605773926),
 ('components', 'working', 4.236789703369141),
 ('containing', 'working', 4.922369956970215),
 ('coordinate', 'working', 5.317715644836426),
 ('data', 'working', 1.472557544708252),
 ('dataset', 'working', 20.869976043701172),
 ('datasets', 'working', 5.506537437438965),
 ('described', 'working', 3.599013090133667),
 ('dimensionality', 'working', 6.837431907653809),
 ('dimensions', 'working', 2.167309045791626),
 ('dimensionsfeatures', 'working', 3.37419056892395),
 ('enabling', 'working', 1.7266998291015625),
 ('fewer', 'working', 2.6665048599243164),
 ('fields', 'working'

In [28]:
print("distance between document1 and document2 is: ",round(distance_similarity1,2))
print("distance between document1 and document3 is: ",round(distance_similarity2,2))

distance between document1 and document2 is:  125.51
distance between document1 and document3 is:  314.4


# Approach 2
### Calculate the Centroids for each document. 

In [29]:
def get_centroid(document):
    centroids=[]
    for word in document:
        centroids.append((document[word][0],document[word][1]))
    return np.mean(centroids, axis=0)

In [30]:
centroids_doc1=get_centroid(unique_word_dict1)
centroids_doc2=get_centroid(unique_word_dict2)
centroids_doc3=get_centroid(unique_word_dict3)
print("centroids document 1: ",centroids_doc1)
print("centroids document 2: ",centroids_doc2)
print("centroids document 3: ",centroids_doc3)

centroids document 1:  [ 3.2327747 -1.4196463]
centroids document 2:  [1.4567492 1.5864433]
centroids document 3:  [-8.263576  -1.8421358]


### calculate cosine similarity between (doc1 and doc2) | and between (doc1 and doc3)  

In [31]:
# compute cosine similarity
similarity1 = np.dot(centroids_doc1,centroids_doc2)/(norm(centroids_doc1)*norm(centroids_doc2))
similarity2 = np.dot(centroids_doc1,centroids_doc3)/(norm(centroids_doc1)*norm(centroids_doc3))
print("similarity between document1 and document2 is: ",round(similarity1,2))
print("similarity between document1 and document3 is: ",round(similarity2,2))

similarity between document1 and document2 is:  0.32
similarity between document1 and document3 is:  -0.81


### Calculate Euclidean distance between (doc1 and doc2) | and between (doc1 and doc3)

In [32]:
# compute cosine similarity
distance1 =distance.euclidean(centroids_doc1,centroids_doc2)
distance2 = distance.euclidean(centroids_doc1,centroids_doc3)
print("distance between document1 and document2 is: ",round(distance1,2))
print("distance between document1 and document3 is: ",round(distance2,2))

distance between document1 and document2 is:  3.49
distance between document1 and document3 is:  11.5
